<a href="https://colab.research.google.com/github/jeosol/langchain-study/blob/main/samw-yt/LangChain_Basics_01_LLMs_%2B_Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langchain: The basics



<img src="https://dl.dropboxusercontent.com/s/b33t7ivjx6oture/YT-cover-vid%231-edited2.png" alt="LangChain Basics" width=600>

In [1]:
!pip -q install openai langchain huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.6 MB/s eta 0:00:00


In [19]:
import os

os.environ['OPENAI_API_KEY'] = '''
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''

## Plain Conditional Generation

### First with OpenAI GPT3

In [20]:
from langchain.llms import OpenAI

In [21]:
llm = OpenAI(model_name='text-davinci-003',
             temperature=0.9,
             max_tokens = 256)

In [22]:
text = "Why did the duck cross the road?"

print(llm(text))



To get to the other side.


### Now with T5-Flan-XL

In [10]:
from langchain.llms import HuggingFaceHub

In [11]:

llm_hf = HuggingFaceHub(
    repo_id="google/flan-t5-xl",
    model_kwargs={"temperature":0.9 }
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [23]:
text = "Why did the chicken cross the road?"

print(llm_hf(text))

KeyboardInterrupt: ignored

## Prompt Templates

In [24]:
from langchain import PromptTemplate


restaurant_template = """
I want you to act as a naming consultant for new restaurants.

Return a list of restaurant names. Each name should be short, catchy and easy to remember. It shoud relate to the type of restaurant you are naming.

What are some good names for a restaurant that is {restaurant_desription}?
"""

prompt = PromptTemplate(
    input_variables=["restaurant_desription"],
    template=restaurant_template,
)

In [26]:
# An example prompt with one input variable
prompt_template = PromptTemplate(input_variables=["restaurant_desription"], template=restaurant_template)


In [30]:
description = "a Greek place that serves fresh lamb souvlakis and other Greek food "
description = "A Nigerian restaurant in texas selling african food "
description_02 = "a burger place that is themed with baseball memorabilia"
description_03 = "a cafe that has live hard rock music and memorabilia"

## to see what the prompt will be like
prompt_template.format(restaurant_desription=description)

'\nI want you to act as a naming consultant for new restaurants.\n\nReturn a list of restaurant names. Each name should be short, catchy and easy to remember. It shoud relate to the type of restaurant you are naming.\n\nWhat are some good names for a restaurant that is A Nigerian restaurant in texas selling african food ?\n'

In [33]:
## querying the model with the prompt template
from langchain.chains import LLMChain


chain = LLMChain(llm=llm, prompt=prompt_template)

# Run the chain only specifying the input variable.
# We are inserting the different restaurant description into the prompt tempate
print(chain.run(description))


1. Lone Star African Grill 
2. Texan Spice Kitchen 
3. Tastes of Nigeria 
4. West African Bistro 
5. The Yoruba Cafe 
6. The Igbo Table 
7. The Suya Spot 
8. The Lagos Bowl 
9. African Kitchen Express 
10. Jollof Junction


## with Few Shot Learning

In [36]:
from langchain import PromptTemplate, FewShotPromptTemplate


In [37]:
# First, create the list of few shot examples.
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]

In [38]:
# Next, we specify the template to format the examples we have provided.
# We use the `PromptTemplate` class for this.
example_formatter_template = """
Word: {word}
Antonym: {antonym}\n
"""
example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)

In [39]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Give the antonym of every input",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="Word: {input}\nAntonym:",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n\n",
)


In [40]:

# We can now generate a prompt using the `format` method.
print(few_shot_prompt.format(input="big"))

Give the antonym of every input


Word: happy
Antonym: sad




Word: tall
Antonym: short



Word: big
Antonym:


In [48]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=few_shot_prompt)

# Run the chain only specifying the input variable.
print(chain.run("This is a good"))

 This is a bad
